
# Programa Ejemplo para realizar el procesamiento de variables categóricas a partir de la técnica One-Hot encoding.

## Etapa 1: Definición del caso de estudio 

Para este ejercicio usaremos la base de datos [tic-tac-toe](https://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame). La idea es predecir el ganador en el juego de tic-tac-toe


In [1]:
# Carga de librerías y la base de datos

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Se cargan los datos
url="https://archive.ics.uci.edu/ml/machine-learning-databases/tic-tac-toe/tic-tac-toe.data"
data = pd.read_csv(url, header=None, na_values=" ?")
data.columns = ["top-left-square", "top-middle-square", "top-right-square",
         "middle-left-square", "middle-middle-square", "middle-right-square",
         "bottom-left-square", "bottom-middle-square", "bottom-right-square",
         "Class"]
data

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,Class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,negative
954,o,x,o,x,x,o,x,o,x,negative
955,o,x,o,x,o,x,x,o,x,negative
956,o,x,o,o,x,x,x,o,x,negative


Se puede notar que todos los atributos son de tipo categórico.

In [2]:
from sklearn.preprocessing import LabelEncoder 

# Se convierten los atributos categóricos a valores numéricos
labelencoder= LabelEncoder() 
for name in data.columns:
  data[name] = labelencoder.fit_transform(data[name])
data.head()

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square,Class
0,2,2,2,2,1,1,2,1,1,1
1,2,2,2,2,1,1,1,2,1,1
2,2,2,2,2,1,1,1,1,2,1
3,2,2,2,2,1,1,1,0,0,1
4,2,2,2,2,1,1,0,1,0,1


In [3]:
# Se dividen los datos en los conjuntos de entrenamiento y prueba. 
# Se usa una división 70/30. 

X_train, X_test, y_train, y_test = train_test_split(data.drop('Class', axis=1), data.Class, test_size=0.3, random_state=123)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(670, 9) (670,)
(288, 9) (288,)


# Etapa 2: Entrenamiento de un sistema de aprendizaje automático usando los datos originales.

Se entrena un clasificador basado en regresión logística sobre la base de datos original. Noten que el único proceso que se aplicó fue convertir los caracteres en números. 

In [4]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

print(classification_report(clf.predict(X_test),y_test))

              precision    recall  f1-score   support

           0       0.73      0.82      0.78        97
           1       0.91      0.85      0.88       191

    accuracy                           0.84       288
   macro avg       0.82      0.84      0.83       288
weighted avg       0.85      0.84      0.84       288



In [5]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)


print(classification_report(logisticRegr.predict(X_test),y_test))

              precision    recall  f1-score   support

           0       0.18      0.65      0.29        31
           1       0.94      0.65      0.77       257

    accuracy                           0.65       288
   macro avg       0.56      0.65      0.53       288
weighted avg       0.86      0.65      0.72       288



Se nota que el rendimiento del clasificador no es satsfactorio. Con el fin de mejorar dicho rendimiento se emplea una codificación de los atributos categórcios.

# Etapa 3: Usar one-hot encoding para procesar los atributos de tipo categórico

La idea es codificar los datos categóricos para que favorezcan el rendimiento del clasificador. Una alternativa es usar la codificacion One-Hot. Esta consiste en lo siguiente.

Suponer que se tiene un atributo que puede tomar uno de 4 valores, digamos 1,2,3,4. La codificación se realiza a partir de un vector binario de 4 posiciones (tantas posiciones como valores pueda tomar el atributo). Así

Categoría 1: [1 0 0 0]

Categoría 2: [0 1 0 0]

Categoría 3: [0 0 1 0]

Categoría 4: [0 0 0 1]


In [6]:
# Ejemplo ilustrativo 
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
X = [[1], [3], [2]]
X1 = enc.fit_transform(X).toarray()
print(X)
print(X1)

[[1], [3], [2]]
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [7]:
# Se aplica la codificación One-Hot a los datos de training
enc.fit(X_train)
X_train_1 = enc.transform(X_train).toarray()
print(X_train_1.shape)
# La base de datos tiene 9 atributos, donde cada uno puede tomar uno de 3 valores.
# En este sentido, dentro de la codificación One-hot, cada atributo se codifica a 
# partir de un vector de 3 posiciones. Así, la base de datos codificada tiene 27
# atributos. 


(670, 27)


In [8]:
# Se entrena el modelo con los datos codificados
logisticRegr_1 = LogisticRegression()
logisticRegr_1.fit(X_train_1, y_train)

LogisticRegression()

In [9]:
# La misma codificación aplicada sobre el conjunto de entrenamiento se aplica sobre
# los datos de prueba. Luego, se verifica el funcionamiento del sistema de aprendizaje
X_test_1 = enc.transform(X_test).toarray()

print(classification_report(logisticRegr_1.predict(X_test_1),y_test))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98       104
           1       1.00      0.97      0.99       184

    accuracy                           0.98       288
   macro avg       0.98      0.99      0.98       288
weighted avg       0.98      0.98      0.98       288



Es evidente que la codificación One-Hot mejora la representación de los atributos categóricos y así incrementa todas las métricas del clasificador.

In [10]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train_1, y_train)

print(classification_report(clf.predict(X_test_1),y_test))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       106
           1       0.98      0.96      0.97       182

    accuracy                           0.96       288
   macro avg       0.96      0.96      0.96       288
weighted avg       0.96      0.96      0.96       288

